In [1]:
domain = 'https://fbref.com'
br = '/pt/'
pais = 'Brasil'
liga = 'Serie A'

url_brasileirao = 'https://fbref.com/pt/comps/24/Serie-A-Stats'

In [2]:
domain+"/{lg}/".format(lg=br[1:-1])

'https://fbref.com/pt/'

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import time

In [4]:
def get_url(html):
    try:
        resp = html.find("a", href=True)['href']
    except:
        resp = ''
#     print(html)  
#     print(resp)
    return resp

In [5]:
def get_equipes(url_competion, url, pais='Brasil', liga='Serie A'):
    r = requests.get(
        #url+"/{lg}/".format(lg=language)
        url_competion
    )
    
    soup = BeautifulSoup(r.text, 'lxml')
    
    tbl = soup.find("table", {"class": "stats_table"})
    tbl_h = tbl.find("thead")
    tbl_b = tbl.find("tbody")
    
    table_rows = tbl_b.find_all("tr")
    
    res = []

    for tr in table_rows:
        td = tr.find_all('td')
        a = td[0].find("a", href=True)
        res.append(
            [pais, liga, a.text, url+a['href']]
        )
        
    return pd.DataFrame(
        res
        , columns=['Pais', 'Liga', 'Equipe', 'link']
    ).assign(
        **{'id': lambda x: x['link'].apply(
            lambda y: y.split('/')[-2]
        )}
    )

In [6]:
parm = {
    'url_competion': 'https://fbref.com/pt/comps/24/Serie-A-Stats'
    , 'url': 'https://fbref.com'
    , 'pais': 'Brasil'
    , 'liga': 'Serie A'
}

df_equipes = get_equipes(**parm)

with open('equipes.json', 'w', encoding='utf-8') as f:
    json.dump(df_equipes.to_dict(orient='records'), f, ensure_ascii=False, indent=4)

In [6]:
def get_partidas_equipe(domain, url, equipe_id):
    r = requests.get(
        url
    )
    
    soup = BeautifulSoup(r.text, 'lxml')
    
    tbl = soup.find("table", id="matchlogs_for")
    tbl_h = tbl.find("thead")
    tbl_b = tbl.find("tbody")
    
    fields = [th.text.strip() for th in tbl_h.find_all('th')]
    
    res = []
    
    for tr in tbl_b.find_all('tr'):
        res.append(
            [tr.find("th").text.strip()]
            + [
                domain + get_url(td) if i in [8, 16] else td.text.strip() for i, td in enumerate(tr.find_all('td'))
            ]
        )
        
    return pd.DataFrame(
        res
        , columns=fields
    ).assign(
        **{'id':lambda x: x['Relatório da Partida'].apply(
            lambda y: y.split('/')[-2]
        )}
        , **{'Data': lambda x: pd.to_datetime(x['Data'])}
        , **{'fgk_equipe': equipe_id}
        , **{'Oponente': lambda x: x['Oponente'].apply(
            lambda y: y.split('/')[-2]
        )}
    )

In [8]:
df_partidas = pd.DataFrame()
for row in df_equipes.iterrows():
    df_partidas = pd.concat(
        [
            df_partidas
            , get_partidas_equipe(
                domain
                , row[1]['link']
                , row[1]['id']
            )
        ]
        , ignore_index=True
    )

In [9]:
df_partidas = pd.read_excel('partidas.xlsx')

In [8]:
!pip install openpyxl

  Using cached openpyxl-3.0.10-py2.py3-none-any.whl (242 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


In [29]:
df_partidas.to_excel('partidas.xlsx')

In [10]:
df_partidas.to_csv('partidas.csv')

In [11]:
df_partidas

,Unnamed: 0,Data,Horário,Camp.,Rodada,Dia,Local,Resultado,GP,GC,...,xGA,Posse,Público,Capitão,Formação,Árbitro,Relatório da Partida,Notas,id,fgk_equipe
0,0,2022-04-06,20:00,Libertadores,Estágio de grupos,qua,Visitante,V,4,0,...,0.9,50.0,NaN,Gustavo Gómez,4-3-3,Darío Herrera,https://fbref.com/pt/partidas/0c2b92d0/Deporti...,NaN,0c2b92d0,abdce579
1,1,2022-04-09,21:00,Série A,Rodada da semana 1,sáb,Em casa,D,2,3,...,1.5,56.0,27.100,Gustavo Gómez,4-3-3,Caio Max Augusto Vieira,https://fbref.com/pt/partidas/c90739cb/Palmeir...,NaN,c90739cb,abdce579
2,2,2022-04-12,21:30,Libertadores,Estágio de grupos,ter,Em casa,V,8,1,...,0.1,76.0,26.409,Gustavo Gómez,4-2-3-1,Derlis López,https://fbref.com/pt/partidas/9cdd9f2a/Palmeir...,NaN,9cdd9f2a,abdce579
3,3,2022-04-16,16:30,Série A,Rodada da semana 2,sáb,Visitante,E,1,1,...,0.8,75.0,12.864,Gustavo Gómez,4-2-3-1,Braulio da Silva Machado,https://fbref.com/pt/partidas/449b7640/Goias-P...,NaN,449b7640,abdce579
4,4,2022-04-20,19:30,Série A,Rodada da semana 4,qua,Visitante,E,0,0,...,1.4,45.0,69.997,Gustavo Gómez,4-2-3-1,Wilton Sampaio,https://fbref.com/pt/partidas/572a91c6/Flameng...,NaN,572a91c6,abdce579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906,906,2022-10-27,19:30,Série A,Rodada da semana 34,qui,Visitante,D,0,1,...,2.0,40.0,NaN,Rodrigo Alves Soares,4-2-3-1,Savio Pereira,https://fbref.com/pt/partidas/ef3e1315/Atletic...,NaN,ef3e1315,d081b697
907,907,2022-11-02,19:00,Série A,Rodada da semana 35,qua,Em casa,D,0,1,...,1.1,39.0,NaN,Rodrigo Alves Soares,4-2-3-1,Marcelo de Lima Henrique,https://fbref.com/pt/partidas/5e918bd2/Juventu...,NaN,5e918bd2,d081b697
908,908,2022-11-05,19:00,Série A,Rodada da semana 36,sáb,Visitante,D,0,1,...,0.7,49.0,NaN,Chico,4-2-3-1,Edina Alves Batista,https://fbref.com/pt/partidas/35877f17/Goias-J...,NaN,35877f17,d081b697
909,909,2022-11-09,21:30,Série A,Rodada da semana 37,qua,Em casa,E,2,2,...,2.4,37.0,NaN,Chico,4-2-3-1,Rodrigo José Pereira de Lima,https://fbref.com/pt/partidas/2984a8ec/Juventu...,NaN,2984a8ec,d081b697


In [32]:
with open('partidas.json', 'w', encoding='utf-8') as f:
    json.dump(df_partidas.to_dict(orient='records'), f, ensure_ascii=False, indent=4)

TypeError: Object of type Timestamp is not JSON serializable

In [12]:
def get_estatisticas_jogador(url, partida_id, casa, visitante):
    r = requests.get(
        url
    )
    
    soup = BeautifulSoup(r.text, 'lxml')
    
    # jogadores de casa
    try:
        tbl = soup.find("table", id="stats_{team_id}_summary".format(team_id=casa))
        tbl_h = tbl.find("thead")
        tbl_b = tbl.find("tbody")

        fields = [a.text.strip() for a in tbl_h.find_all('tr')[1].find_all('th')]

        res = []

        for tr in tbl_b.find_all('tr'):
            res.append(
                [get_url(tr.find("th"))]
                + [get_url(a) if i == 1 else a.text.strip() for i, a in enumerate(tr.find_all('td'))]
            )
    
        df1 = pd.DataFrame(
            res
            , columns=fields
        ).assign(
            **{'Jogador': lambda x: x['Jogador'].apply(
                lambda y: y.split('/')[-2] if len(y.split('/')) > 1 else y
            )}
            , **{'Nação': lambda x: x['Nação'].apply(
                lambda y: y.split('/')[-2] if len(y.split('/')) > 1 else y
            )}
            , **{'fkg_equipe':casa}
            , **{'fgk_partida': partida_id}
        )
    except:
        df1 = pd.DataFrame()
        print('url', url, 'partida_id', partida_id, 'casa', casa, 'visitante', visitante)
        print('Erro nos jogadores de casa')
        
    # jogadores visitante
    try:
        tbl = soup.find("table", id="stats_{team_id}_summary".format(team_id=visitante))
        tbl_h = tbl.find("thead")
        tbl_b = tbl.find("tbody")

        fields = [a.text.strip() for a in tbl_h.find_all('tr')[1].find_all('th')]

        res = []

        for tr in tbl_b.find_all('tr'):
            res.append(
                [get_url(tr.find("th"))]
                + [url+get_url(a) if i == 1 else a.text.strip() for i, a in enumerate(tr.find_all('td'))]
            )

        df2 = pd.DataFrame(
            res
            , columns=fields
        ).assign(
            **{'Jogador': lambda x: x['Jogador'].apply(
                lambda y: y.split('/')[-2] if len(y.split('/')) > 1 else y
            )}
            , **{'Nação': lambda x: x['Nação'].apply(
                lambda y: y.split('/')[-2] if len(y.split('/')) > 1 else y
            )}
            , **{'fkg_equipe':visitante}
            , **{'fgk_partida': partida_id}
        )
    except:
        df2 = pd.DataFrame()
        print('url', url, 'partida_id', partida_id, 'casa', casa, 'visitante', visitante)
        print('Erro nos jogadores visitantes')
    
    # Goleiro de casa
    try:
        tbl = soup.find("table", id="keeper_stats_{team_id}".format(team_id=casa))
        tbl_h = tbl.find("thead")
        tbl_b = tbl.find("tbody")

        fields = [a.text.strip() for a in tbl_h.find_all('tr')[1].find_all('th')]

        res = []

        for tr in tbl_b.find_all('tr'):
            res.append(
                [get_url(tr.find("th"))]
                + [url+get_url(a) if i == 0 else a.text.strip() for i, a in enumerate(tr.find_all('td'))]
            )

        df3 = pd.DataFrame(
            res
            , columns=fields
        ).assign(
            **{'Jogador': lambda x: x['Jogador'].apply(
                lambda y: y.split('/')[-2] if len(y.split('/')) > 1 else y
            )}
            , **{'Nação': lambda x: x['Nação'].apply(
                lambda y: y.split('/')[-2] if len(y.split('/')) > 1 else y
            )}
            , **{'fkg_equipe':casa}
            , **{'fgk_partida': partida_id}
        )
    except:
        df3 = pd.DataFrame()
        print('url', url, 'partida_id', partida_id, 'casa', casa, 'visitante', visitante)
        print('Erro no goleiro de casa')
    
    # Goleiro visitante
    try:
        tbl = soup.find("table", id="keeper_stats_{team_id}".format(team_id=visitante))
        tbl_h = tbl.find("thead")
        tbl_b = tbl.find("tbody")

        fields = [a.text.strip() for a in tbl_h.find_all('tr')[1].find_all('th')]

        res = []

        for tr in tbl_b.find_all('tr'):
            res.append(
                [get_url(tr.find("th"))]
                + [url+get_url(a) if i == 0 else a.text.strip() for i, a in enumerate(tr.find_all('td'))]
            )

        df4 = pd.DataFrame(
            res
            , columns=fields
        ).assign(
            **{'Jogador': lambda x: x['Jogador'].apply(
                lambda y: y.split('/')[-2] if len(y.split('/')) > 1 else y
            )}
            , **{'Nação': lambda x: x['Nação'].apply(
                lambda y: y.split('/')[-2] if len(y.split('/')) > 1 else y
            )}
            , **{'fkg_equipe':visitante}
            , **{'fgk_partida': partida_id}
        )
    except:
        df4 = pd.DataFrame()
        print('url', url, 'partida_id', partida_id, 'casa', casa, 'visitante', visitante)
        print('Erro no goleiro visitante')
        
    return pd.concat([df1, df2], ignore_index=True), pd.concat([df3, df4], ignore_index=True)

In [25]:
df_linha = pd.DataFrame()
df_gol = pd.DataFrame()
i = 50
print(i, 'de', len(df_partidas))
if not (df_partidas.iloc[i]['Resultado'] == ''):
    if df_partidas.iloc[i]['Local'] == 'Em casa':
        casa = df_partidas.iloc[i]['fgk_equipe']
        visitante = df_partidas.iloc[i]['Oponente']
    else:
        casa = df_partidas.iloc[i]['Oponente']
        visitante = df_partidas.iloc[i]['fgk_equipe']

    df_linha_t, df_gol_t = get_estatisticas_jogador(
        df_partidas.iloc[i]['Relatório da Partida']
        , df_partidas.iloc[i]['id']
        , casa
        , visitante
    )

    df_linha = pd.concat(
        [df_linha, df_linha_t]
        , ignore_index=True
    )

    df_gol = pd.concat(
        [df_gol, df_gol_t]
        , ignore_index=True
    )

50 de 911
url https://fbref.com/pt/partidas/8299b04d/Club-9-de-Octubre-Internacional-2022Abril6-Copa-Sudamericana partida_id 8299b04d casa f7dac7a4 visitante 6f7e1f03
Erro nos jogadores de casa
url https://fbref.com/pt/partidas/8299b04d/Club-9-de-Octubre-Internacional-2022Abril6-Copa-Sudamericana partida_id 8299b04d casa f7dac7a4 visitante 6f7e1f03
Erro nos jogadores visitantes
url https://fbref.com/pt/partidas/8299b04d/Club-9-de-Octubre-Internacional-2022Abril6-Copa-Sudamericana partida_id 8299b04d casa f7dac7a4 visitante 6f7e1f03
Erro no goleiro de casa
url https://fbref.com/pt/partidas/8299b04d/Club-9-de-Octubre-Internacional-2022Abril6-Copa-Sudamericana partida_id 8299b04d casa f7dac7a4 visitante 6f7e1f03
Erro no goleiro visitante


In [14]:
df_linha = pd.DataFrame()
df_gol = pd.DataFrame()

c = 0

for row in df_partidas.iterrows():
    print(c, 'de', len(df_partidas))
#    if c%10 == 0:
#         time.sleep(10)
        
        
    if not (row[1]['Resultado'] == ''):
        if row[1]['Local'] == 'Em casa':
            casa = row[1]['fgk_equipe']
            visitante = row[1]['Oponente']
        else:
            casa = row[1]['Oponente']
            visitante = row[1]['fgk_equipe']

        df_linha_t, df_gol_t = get_estatisticas_jogador(
            row[1]['Relatório da Partida']
            , row[1]['id']
            , casa
            , visitante
        )

        df_linha = pd.concat(
            [df_linha, df_linha_t]
            , ignore_index=True
        )

        df_gol = pd.concat(
            [df_gol, df_gol_t]
            , ignore_index=True
        )
    c += 1

0 de 911
1 de 911
2 de 911
3 de 911
4 de 911
5 de 911
6 de 911
7 de 911
8 de 911
9 de 911
10 de 911
11 de 911
12 de 911
13 de 911
14 de 911
15 de 911
16 de 911
17 de 911
18 de 911
19 de 911
20 de 911
21 de 911
22 de 911
23 de 911
24 de 911
25 de 911
26 de 911
27 de 911
28 de 911
29 de 911
30 de 911
url https://fbref.com/pt/partidas/47126b45/Palmeiras-Atletico-Mineiro-2022Agosto10-Copa-Libertadores partida_id 47126b45 casa abdce579 visitante 422bb734
Erro nos jogadores de casa
url https://fbref.com/pt/partidas/47126b45/Palmeiras-Atletico-Mineiro-2022Agosto10-Copa-Libertadores partida_id 47126b45 casa abdce579 visitante 422bb734
Erro nos jogadores visitantes
url https://fbref.com/pt/partidas/47126b45/Palmeiras-Atletico-Mineiro-2022Agosto10-Copa-Libertadores partida_id 47126b45 casa abdce579 visitante 422bb734
Erro no goleiro de casa
url https://fbref.com/pt/partidas/47126b45/Palmeiras-Atletico-Mineiro-2022Agosto10-Copa-Libertadores partida_id 47126b45 casa abdce579 visitante 422bb734
Err

url https://fbref.com/pt/partidas/e215127f/Palmeiras-Sao-Paulo-2022Outubro16-Serie-A partida_id e215127f casa abdce579 visitante 5f232eb1
Erro nos jogadores de casa
url https://fbref.com/pt/partidas/e215127f/Palmeiras-Sao-Paulo-2022Outubro16-Serie-A partida_id e215127f casa abdce579 visitante 5f232eb1
Erro nos jogadores visitantes
url https://fbref.com/pt/partidas/e215127f/Palmeiras-Sao-Paulo-2022Outubro16-Serie-A partida_id e215127f casa abdce579 visitante 5f232eb1
Erro no goleiro de casa
url https://fbref.com/pt/partidas/e215127f/Palmeiras-Sao-Paulo-2022Outubro16-Serie-A partida_id e215127f casa abdce579 visitante 5f232eb1
Erro no goleiro visitante
44 de 911
url https://fbref.com/pt/partidas/d16344ec/Palmeiras-Avai-2022Outubro22-Serie-A partida_id d16344ec casa abdce579 visitante f205258a
Erro nos jogadores de casa
url https://fbref.com/pt/partidas/d16344ec/Palmeiras-Avai-2022Outubro22-Serie-A partida_id d16344ec casa abdce579 visitante f205258a
Erro nos jogadores visitantes
url http

url https://fbref.com/pt/partidas/8f07c6b0/Independiente-Internacional-2022Abril26-Copa-Sudamericana partida_id 8f07c6b0 casa 70068101 visitante 6f7e1f03
Erro nos jogadores de casa
url https://fbref.com/pt/partidas/8f07c6b0/Independiente-Internacional-2022Abril26-Copa-Sudamericana partida_id 8f07c6b0 casa 70068101 visitante 6f7e1f03
Erro nos jogadores visitantes
url https://fbref.com/pt/partidas/8f07c6b0/Independiente-Internacional-2022Abril26-Copa-Sudamericana partida_id 8f07c6b0 casa 70068101 visitante 6f7e1f03
Erro no goleiro de casa
url https://fbref.com/pt/partidas/8f07c6b0/Independiente-Internacional-2022Abril26-Copa-Sudamericana partida_id 8f07c6b0 casa 70068101 visitante 6f7e1f03
Erro no goleiro visitante
56 de 911
url https://fbref.com/pt/partidas/ce0d4b19/Internacional-Avai-2022Maio1-Serie-A partida_id ce0d4b19 casa 6f7e1f03 visitante f205258a
Erro nos jogadores de casa
url https://fbref.com/pt/partidas/ce0d4b19/Internacional-Avai-2022Maio1-Serie-A partida_id ce0d4b19 casa 6f

url https://fbref.com/pt/partidas/93d7983b/Internacional-Botafogo-RJ-2022Junho19-Serie-A partida_id 93d7983b casa 6f7e1f03 visitante d9fdd9d9
Erro nos jogadores de casa
url https://fbref.com/pt/partidas/93d7983b/Internacional-Botafogo-RJ-2022Junho19-Serie-A partida_id 93d7983b casa 6f7e1f03 visitante d9fdd9d9
Erro nos jogadores visitantes
url https://fbref.com/pt/partidas/93d7983b/Internacional-Botafogo-RJ-2022Junho19-Serie-A partida_id 93d7983b casa 6f7e1f03 visitante d9fdd9d9
Erro no goleiro de casa
url https://fbref.com/pt/partidas/93d7983b/Internacional-Botafogo-RJ-2022Junho19-Serie-A partida_id 93d7983b casa 6f7e1f03 visitante d9fdd9d9
Erro no goleiro visitante
69 de 911
url https://fbref.com/pt/partidas/ed828cae/Internacional-Coritiba-2022Junho24-Serie-A partida_id ed828cae casa 6f7e1f03 visitante d680d257
Erro nos jogadores de casa
url https://fbref.com/pt/partidas/ed828cae/Internacional-Coritiba-2022Junho24-Serie-A partida_id ed828cae casa 6f7e1f03 visitante d680d257
Erro nos j

url https://fbref.com/pt/partidas/75d4fb28/Internacional-Fluminense-2022Agosto14-Serie-A partida_id 75d4fb28 casa 6f7e1f03 visitante 84d9701c
Erro nos jogadores de casa
url https://fbref.com/pt/partidas/75d4fb28/Internacional-Fluminense-2022Agosto14-Serie-A partida_id 75d4fb28 casa 6f7e1f03 visitante 84d9701c
Erro nos jogadores visitantes
url https://fbref.com/pt/partidas/75d4fb28/Internacional-Fluminense-2022Agosto14-Serie-A partida_id 75d4fb28 casa 6f7e1f03 visitante 84d9701c
Erro no goleiro de casa
url https://fbref.com/pt/partidas/75d4fb28/Internacional-Fluminense-2022Agosto14-Serie-A partida_id 75d4fb28 casa 6f7e1f03 visitante 84d9701c
Erro no goleiro visitante
82 de 911
url https://fbref.com/pt/partidas/f7bcfeff/Avai-Internacional-2022Agosto22-Serie-A partida_id f7bcfeff casa f205258a visitante 6f7e1f03
Erro nos jogadores de casa
url https://fbref.com/pt/partidas/f7bcfeff/Avai-Internacional-2022Agosto22-Serie-A partida_id f7bcfeff casa f205258a visitante 6f7e1f03
Erro nos jogador

url https://fbref.com/pt/partidas/5b8271d0/Internacional-Ceara-2022Outubro26-Serie-A partida_id 5b8271d0 casa 6f7e1f03 visitante 2f335e17
Erro nos jogadores de casa
url https://fbref.com/pt/partidas/5b8271d0/Internacional-Ceara-2022Outubro26-Serie-A partida_id 5b8271d0 casa 6f7e1f03 visitante 2f335e17
Erro nos jogadores visitantes
url https://fbref.com/pt/partidas/5b8271d0/Internacional-Ceara-2022Outubro26-Serie-A partida_id 5b8271d0 casa 6f7e1f03 visitante 2f335e17
Erro no goleiro de casa
url https://fbref.com/pt/partidas/5b8271d0/Internacional-Ceara-2022Outubro26-Serie-A partida_id 5b8271d0 casa 6f7e1f03 visitante 2f335e17
Erro no goleiro visitante
94 de 911
url https://fbref.com/pt/partidas/64a00412/America-MG-Internacional-2022Novembro2-Serie-A partida_id 64a00412 casa 1f68d780 visitante 6f7e1f03
Erro nos jogadores de casa
url https://fbref.com/pt/partidas/64a00412/America-MG-Internacional-2022Novembro2-Serie-A partida_id 64a00412 casa 1f68d780 visitante 6f7e1f03
Erro nos jogadores

url https://fbref.com/pt/partidas/8a2778e4/Fluminense-Internacional-2022Abril23-Serie-A partida_id 8a2778e4 casa 84d9701c visitante 6f7e1f03
Erro nos jogadores de casa
url https://fbref.com/pt/partidas/8a2778e4/Fluminense-Internacional-2022Abril23-Serie-A partida_id 8a2778e4 casa 84d9701c visitante 6f7e1f03
Erro nos jogadores visitantes
url https://fbref.com/pt/partidas/8a2778e4/Fluminense-Internacional-2022Abril23-Serie-A partida_id 8a2778e4 casa 84d9701c visitante 6f7e1f03
Erro no goleiro de casa
url https://fbref.com/pt/partidas/8a2778e4/Fluminense-Internacional-2022Abril23-Serie-A partida_id 8a2778e4 casa 84d9701c visitante 6f7e1f03
Erro no goleiro visitante
107 de 911
url https://fbref.com/pt/partidas/7d8c32ca/Fluminense-CA-Union-2022Abril26-Copa-Sudamericana partida_id 7d8c32ca casa 84d9701c visitante 5adc7e67
Erro nos jogadores de casa
url https://fbref.com/pt/partidas/7d8c32ca/Fluminense-CA-Union-2022Abril26-Copa-Sudamericana partida_id 7d8c32ca casa 84d9701c visitante 5adc7e67

url https://fbref.com/pt/partidas/af8dcb20/America-MG-Fluminense-2022Junho15-Serie-A partida_id af8dcb20 casa 1f68d780 visitante 84d9701c
Erro nos jogadores de casa
url https://fbref.com/pt/partidas/af8dcb20/America-MG-Fluminense-2022Junho15-Serie-A partida_id af8dcb20 casa 1f68d780 visitante 84d9701c
Erro nos jogadores visitantes
url https://fbref.com/pt/partidas/af8dcb20/America-MG-Fluminense-2022Junho15-Serie-A partida_id af8dcb20 casa 1f68d780 visitante 84d9701c
Erro no goleiro de casa
url https://fbref.com/pt/partidas/af8dcb20/America-MG-Fluminense-2022Junho15-Serie-A partida_id af8dcb20 casa 1f68d780 visitante 84d9701c
Erro no goleiro visitante
120 de 911
url https://fbref.com/pt/partidas/f2120467/Fluminense-Avai-2022Junho19-Serie-A partida_id f2120467 casa 84d9701c visitante f205258a
Erro nos jogadores de casa
url https://fbref.com/pt/partidas/f2120467/Fluminense-Avai-2022Junho19-Serie-A partida_id f2120467 casa 84d9701c visitante f205258a
Erro nos jogadores visitantes
url https

url https://fbref.com/pt/partidas/3b1c4ae4/Atletico-Paranaense-Fluminense-2022Setembro3-Serie-A partida_id 3b1c4ae4 casa 2091c619 visitante 84d9701c
Erro nos jogadores de casa
url https://fbref.com/pt/partidas/3b1c4ae4/Atletico-Paranaense-Fluminense-2022Setembro3-Serie-A partida_id 3b1c4ae4 casa 2091c619 visitante 84d9701c
Erro nos jogadores visitantes
url https://fbref.com/pt/partidas/3b1c4ae4/Atletico-Paranaense-Fluminense-2022Setembro3-Serie-A partida_id 3b1c4ae4 casa 2091c619 visitante 84d9701c
Erro no goleiro de casa
url https://fbref.com/pt/partidas/3b1c4ae4/Atletico-Paranaense-Fluminense-2022Setembro3-Serie-A partida_id 3b1c4ae4 casa 2091c619 visitante 84d9701c
Erro no goleiro visitante
133 de 911
url https://fbref.com/pt/partidas/6797de50/Fluminense-Fortaleza-2022Setembro10-Serie-A partida_id 6797de50 casa 84d9701c visitante a9d0ab0e
Erro nos jogadores de casa
url https://fbref.com/pt/partidas/6797de50/Fluminense-Fortaleza-2022Setembro10-Serie-A partida_id 6797de50 casa 84d9701

KeyboardInterrupt: 

In [ ]:
# df_equipes
# df_partidas
# df_linha
# df_gol

In [ ]:
df_partidas.query("fgk_equipe == 'abdce579'").iloc[35]['Resultado'] == ''

In [ ]:
link = df_partidas.iloc[0]['Relatório da Partida']
link

In [ ]:
partida_id = df_partidas.iloc[0]['id']
partida_id

In [ ]:
casa = df_partidas.iloc[0]['Oponente']
casa

In [ ]:
visitante = df_partidas.iloc[0]['fgk_equipe']
visitante

In [ ]:


r = requests.get(
    link
)
soup = BeautifulSoup(r.text, 'lxml')
tbl = soup.find("table", id="keeper_stats_{team_id}".format(team_id=casa))

tbl_h = tbl.find("thead")
tbl_b = tbl.find("tbody")

fields = [a.text.strip() for a in tbl_h.find_all('tr')[1].find_all('th')]

res = []

for tr in tbl_b.find_all('tr'):
    res.append(
        [tr.find("th").find("a", href=True)['href']]
        + [url+a.find("a", href=True)['href'] if i == 0 else a.text.strip() for i, a in enumerate(tr.find_all('td'))]
    )
    
pd.DataFrame(
    res
    , columns=fields
).assign(
    **{'Jogador': lambda x: x['Jogador'].apply(
        lambda y: y.split('/')[-2]
    )}
    , **{'Nação': lambda x: x['Nação'].apply(
        lambda y: y.split('/')[-2]
    )}
    , **{'fkg_equipe':casa}
    , **{'fgk_partida': partida_id}
)

In [ ]:
pd.DataFrame(res, columns=fields).assign(
    **{'ID':lambda x: x['Relatório da Partida'].apply(
        lambda y: y.split('/')[-2]
    )}
    , **{'Data': lambda x: pd.to_datetime(x['Data'])}
)

In [ ]:
df_equipes.assign(
    **{'id': lambda x: x['link'].apply(
        lambda y: y.split('/')[-2]
    )}
)

In [ ]:
table_rows = tbl_b.find_all("tr")

res = []

for tr in table_rows:
    td = tr.find_all('td')
    a = td[0].find("a", href=True)
    res.append(
        [pais, liga, a.text, url+a['href']]
    )

In [ ]:
df_equipes = pd.DataFrame(res, columns=['Pais', 'Liga', 'Equipe', 'link'])

In [ ]:
df_equipes['link'].iloc[0]

In [ ]:
fields = []
cells = tbl_h.find_all("th")
fields = [cell.text.strip() for cell in cells if cell.text.strip()][1:]

In [ ]:
table_rows = tbl_b.find_all("tr")

res = []

for tr in table_rows:
    td = tr.find_all('td')
#     row = [cell.text.strip() for cell in td if cell.text.strip()]
    row = [cell.text for cell in td if cell.text.strip()]
    if row:
        res.append(row)

In [ ]:
df = pd.DataFrame(res, columns=fields)
df['Equipe']

In [ ]:
tbl_b

In [ ]:
for i, v in enumerate(['a', 'b', 'c']):
    print(i, v)